In [2]:
import requests

def chat_with_ollama(prompt, model="llama3.2:latest"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": True  # set True for streaming responses
    }
    response = requests.post(url, json=payload)
    return response.json()["response"]

if __name__ == "__main__":
    user_input = "Explain me string theory"
    reply = chat_with_ollama(user_input)
    print(f"Ollama: {reply}")

JSONDecodeError: Extra data: line 2 column 1 (char 107)